In [1]:
from sklearn.datasets import load_breast_cancer

In [2]:
data = load_breast_cancer()

In [3]:
len(data.feature_names)

30

In [4]:
data.feature_names

array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype='|S23')

In [5]:
len(data.data[:,2])


569

In [6]:
len(data.data[1,:])

30

In [7]:
min(data.data[:,2])


43.79

In [8]:
sum(data.target)

357

In [9]:
#print(data.data[:5])
#print(data.data[:,3])

In [10]:
import numpy as np

In [11]:
median=np.median(data.data[:,3])
print(median)

551.1


In [12]:
import math
def H(x):
    x=np.array(x)
    n=len(x)
    m=sum(x)
    if(m==0 or m==n):
        return 0
    ans=0
    p=m/n
    ans-=p*math.log(p,2)
    p=1-p
    ans-=p*math.log(p,2)
    return ans

根据各属性的中值划分成二叉树，并由此根据信息增益来判断合适的划分所在的列

In [13]:
def condition_entropy(dataset,axis):
    median=np.median(dataset[:,axis])
    left=[]
    right=[]
    for x in dataset:
        if(x[axis]<median):
            left.append(x)
        else :
            right.append(x)
    if(len(left)==0 or len(right)==0):
        return 0
    n=len(dataset)
    left=np.array(left)
    
    #print(np.shape(left))
    h1=H(left[:,-1])*len(left)/n
    right=np.array(right)
    h2=H(right[:,-1])*len(right)/n
    return h1+h2

In [14]:
def min_condition_entropy_index(dataset,axis_n):
    index=-1
    min_condition_entropy=0
    dataset=np.array(dataset)
    for axis in range(axis_n):
        value=condition_entropy(dataset,axis)
        if(index==-1):
            index=axis
            min_condition_entropy=value
        if(value<min_condition_entropy):
            index=axis
            min_condition_entropy=value
    return index

In [15]:
def divide(dataset,axis):
    dataset=np.array(dataset)
    median=np.median(dataset[:,axis])
    left=[]
    right=[]
    for x in dataset:
        if(x[axis]<median):
            left.append(x)
        else :
            right.append(x)
    return left,right

数据的预处理

In [16]:
def combine(x,y):
    dataset=[]
    n=len(y)
    
    for i in range(n):
        t=[]
        t=np.append(t,x[i])
        t=np.append(t,y[i])
        dataset.append(t)
    return dataset
        
#x=data.data[:5]
#y=data.target[:5]
#print(data.data[:5])
dataset=combine(data.data,data.target)
dataset=np.array(dataset)
#print(dataset[:5])

划分数据集

In [17]:
import random
def split(x,y):
    n=len(x)
    t=[i for i in range(n)]
    random.shuffle(t)
    x=x[t]
    y=y[t]
    m=int(0.7*n)
    train_x=x[:m]
    train_y=y[:m]
    test_x=x[m:]
    test_y=y[m:]
    return train_x,train_y,test_x,test_y

In [18]:
train_x,train_y,test_x,test_y=split(data.data,data.target)

In [19]:
print(len(train_x))

398


In [20]:
dataset=combine(train_x,train_y)


In [21]:
index=min_condition_entropy_index(np.array(dataset),30)

In [22]:
left,right=divide(dataset,index)
np.shape(left)

(199, 31)

In [23]:
len(left)+len(right)

398

In [24]:
class Tree:
    def __init__(self,axis,median,target=-1):
        self.axis=axis
        self.median=median
        self.target=target
        self.left=None
        self.right=None
        

In [25]:
def create_tree(dataset,axis_n=30,min_node=10):
    n=len(dataset)
    dataset=np.array(dataset)
    m=sum(dataset[:,-1])
    
    if(n<min_node):
        if(m>n/2):
            node=Tree(0,0,1);
        else:
            node=Tree(0,0,0);
        return node
    
    
    if(m==0 or n==m):
        node = Tree(0,0,dataset[0,-1])
        return node
    
    axis=min_condition_entropy_index(dataset,axis_n)
    median=np.median(dataset[:,axis])
    node = Tree(axis,median)
    #print(axis,median,len(dataset))
    left,right=divide(dataset,axis)
    if(len(left)>0):
        node.left=create_tree(left,axis_n)
    if(len(right)>0):
        node.right=create_tree(right,axis_n)
    return node

root=create_tree(dataset)


In [26]:
def search(x,axis_n=30):
    node=root
    while (node.target==-1):
        axis=node.axis
        median=node.median
        if(x[axis]<median):
            node=node.left
        else:
            node=node.right
    return node.target

#dataset[:,-1]
